![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
LangGraph 101
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
# install_packages([('markitdown[all]', 'markitdown'),])

In [ ]:
#@title 📂 Dokumente, Bilder { display-mode: "form" }


# 1 | Einleitung: Warum LangGraph?
---



LangGraph ist ein Framework zur Entwicklung **komplexer KI-Agenten**, das Workflows als **gerichtete Graphen** organisiert. LangGraph bietet eine Grundlage für Anwendungen, bei denen KI-Agenten **über mehrere Schritte hinweg Entscheidungen treffen** oder ihren **Zustand verwalten** müssen. Dies ist essenziell für langlebige Systeme, da die Zustände zwischen Schritten **automatisch erhalten** bleiben.

**Kernfunktionen von LangGraph:**

- **Zustandsverwaltung (Stateful):** Automatisches Beibehalten von Kontext über alle Schritte.  
- **Dynamische Entscheidungen:** Agenten können Schleifen und bedingte Verzweigungen nutzen.  
- **Resilienz:** Ermöglicht Self-Correction (Selbstkorrektur) und Tool-Delegation.  



# 2 | LangGraph vs. LangChain
---



LangGraph nutzt die Komponenten von LangChain, verschiebt aber den Fokus von sequenziellen Abläufen hin zu dynamischer Steuerung.

| **Kategorie**      | **LangChain**                                 | **LangGraph**                                                  |
| ------------------ | --------------------------------------------- | -------------------------------------------------------------- |
| **Struktur**       | Lineare Ketten (`Chains`) und Agent-Sequenzen | **Graphbasiert** (Knoten und Kanten).                          |
| **Steuerung**      | Sequenziell, vordefinierter Ablauf            | **Bedingte Steuerung** (Loops und Verzweigungen).              |
| **State Handling** | Manuelle Speicherverwaltung (`Memory`)        | **Automatisches Checkpointing** des gesamten Graphen-Zustands. |
| **Komplexität**    | Einfache Pipelines bis mittlere Agenten       | **Hohe Komplexität:** Adaptive, Multi-Agent-Systeme.           |
| **Kernobjekte**    | `Chain`, `Agent`                           | `StateGraph`, `START`, `END`.                                  |



# 3 | Agentenkonzepte und Graphen-Struktur
---




Nachdem die grundlegenden Unterschiede klar sind, folgt nun der praktische Aufbau eines Workflows. Die Logik der Agenten wird in **Knoten** (`nodes`) gekapselt, und der Fluss wird durch **Kanten** (`edges`) definiert.

**Minimaler Workflow**

Dieses Beispiel demonstriert den Aufbau eines minimalen Workflows mit dem expliziten `START`- und `END`-Knoten.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import InMemorySaver
from IPython.display import Image

# Define the State (Status-Schema)
class AgentState(TypedDict):
    messages: List[str]

# Node Function
def process_message(state: AgentState) -> AgentState:
    last_message = state['messages'][-1]
    response = "Agent: Message received and processed: {}".format(last_message)
    return {"messages": state["messages"] + [response]}

# Graph Construction
builder = StateGraph(AgentState)
builder.add_node("processor", process_message)
builder.add_edge(START, "processor")
builder.add_edge("processor", END)

# Compilation to "runner"
runner = builder.compile(checkpointer=InMemorySaver())
display(Image(runner.get_graph().draw_mermaid_png()))

# Invocation (Start of the Workflow)
response = runner.invoke(
    {"messages": ["User: Start the process."]},
    {"configurable": {"thread_id": "simple_flow_1"}}
)

print(response["messages"])

# 4 | Memory-Management und Checkpointing

LangGraph speichert den gesamten Graphen-Zustand über das **Checkpointer-System**. Eine `thread_id` dient als Schlüssel für die Persistenz.

**Checkpointing-Konzept**

Dieses Beispiel demonstriert, dass eine Information (`note`) im ersten Lauf im State gespeichert wird und im zweiten Lauf unter derselben `thread_id` verfügbar ist.

In [ ]:
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict, List
from IPython.display import Image

# State für das Memory-Beispiel
class SimpleState(TypedDict):
    messages: List[str]
    note: str

# Knoten-Funktion
def store_note(state: SimpleState) -> SimpleState:
    message = state['messages'][-1]
    note = "Note from Run 1: {}".format(message)
    response = "Agent: Note stored."
    return {"messages": state["messages"] + [response], "note": note}

# Graph-Aufbau
builder = StateGraph(SimpleState)
builder.add_node("memory_node", store_note)
builder.add_edge(START, "memory_node")
builder.add_edge("memory_node", END)

# Kompilierung zum "runner"
runner = builder.compile(checkpointer=InMemorySaver())
display(Image(runner.get_graph().draw_mermaid_png()))

# Invokierung (Run 1)
thread_id = "user_memory_session"
run_1 = runner.invoke(
    {"messages": ["User: Remember I like dogs."], "note": ""},
    {"configurable": {"thread_id": thread_id}}
)

print("Run 1 - State Note: {}".format(run_1['note']))

# Invokierung (Run 2: Lädt den gespeicherten State von Run 1)
run_2 = runner.invoke(
    {"messages": ["User: Load state."]},
    {"configurable": {"thread_id": thread_id}}
)

print("\nRun 2 - Geladene Note: {}".format(run_2['note']))

# Hinweis: Das Beispiel nutzt InMemorySaver. Für persistente Speicherung über Prozesse
# hinweg (z. B. nach einem Programmneustart) wäre ein PostgresSaver oder
# SQLAlchemySaver erforderlich.

# 5 | Tool-Integration und Bedingte Steuerung
---




Die **bedingte Kante** (`add_conditional_edges`) ist das Kernelement für die dynamische Steuerung, da sie den Workflow-Pfad basierend auf dem Ergebnis eines Knotens verzweigt.

**Beispiel: Bedingte Tool-Nutzung**

Dieses Beispiel zeigt einen **Router-Knoten**, der den Workflow zur Berechnung oder zur direkten Antwort steuert.

**Schema:** `START` → `router` → ( `math_tool` oder `final_answer` ) → `END`

In [ ]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, List
from IPython.display import Image

# Definiere den State
class ToolState(TypedDict):
    messages: List[str]
    next: str

# Knoten: Router (Routing-Funktion)
def router_node(state: ToolState) -> ToolState:
    last_message = state['messages'][-1].lower()
    if "calculate" in last_message or "summe" in last_message:
        return {"next": "math_tool"} # Return a dictionary with the next node
    return {"next": "final_answer"} # Return a dictionary with the next node

# Knoten: Tool (simuliert eine Berechnung)
def math_tool_node(state: ToolState) -> ToolState:
    response = "Agent: The result of your calculation is 42."
    return {"messages": state["messages"] + [response]}

# Knoten: Direkte Antwort
def direct_response_node(state: ToolState) -> ToolState:
    response = "Agent: I can only process math requests right now."
    return {"messages": state["messages"] + [response]}

# Graph-Konstruktion
builder = StateGraph(ToolState)
builder.add_node("router", router_node)
builder.add_node("math_tool", math_tool_node)
builder.add_node("final_answer", direct_response_node)

builder.add_edge(START, "router")

# Bedingte Kante
builder.add_conditional_edges(
    "router",
    lambda x: x["next"],
    {"math_tool": "math_tool", "final_answer": "final_answer"}
)

# Kanten zum Ende
builder.add_edge("math_tool", END)
builder.add_edge("final_answer", END)

# Kompilierung zum "runner"
runner = builder.compile()
display(Image(runner.get_graph().draw_mermaid_png()))

# Test 1: Tool benötigt
response_tool = runner.invoke({"messages": ["User: Please calculate the sum."]})
print("1. Tool-Pfad: {}".format(response_tool['messages'][-1]))

# Test 2: Direkte Antwort benötigt
response_direct = runner.invoke({"messages": ["User: How are you?"]})
print("\n2. Antwort-Pfad: {}".format(response_direct['messages'][-1]))

# 6 | Anwendungsfälle und Integration
---

<p><font color='black' size="5">
Wann LangGraph sinnvoll ist?
</font></p>


LangGraph eignet sich, wenn der Agent eine komplexe, **nicht-lineare Steuerung** benötigt.

- **Langlebige Chatbots:** Verwaltung des Gesprächszustands über Stunden/Tage hinweg (dank Checkpointing).
    
- **Multi-Agenten-Systeme:** Koordination von Rollen (z. B. Research Agent → Review Agent → Final Answer Agent).
    
- **Workflow-Systeme mit Entscheidungslogik:** Komplexe Support-Automation, bei der Entscheidungen (Tool-Aufruf, Delegation, menschliches Eingreifen) auf Grundlage des State getroffen werden.


<p><font color='black' size="5">
Brücke zu LangChain: `ToolNode`
</font></p>

LangGraph kann nahtlos **LangChain Tools** nutzen. Der `ToolNode` ist ein spezieller LangGraph-Knoten, der automatisch das Input-/Output-Handling eines LangChain-Tools übernimmt.

In [ ]:
from langchain.tools import tool
from langgraph.prebuilt import ToolNode

# Definiere ein LangChain Tool
@tool
def dummy_data_lookup(query: str) -> str:
    """Führt eine Datenabfrage durch und gibt ein Ergebnis zurück."""
    return f"Result for query '{query}': Data found."

# Das ToolNode wird als Knoten in den LangGraph eingefügt:
tool_node = ToolNode([dummy_data_lookup])

# In der Graph-Konstruktion würde nun 'tool_node' verwendet:
# builder.add_node("data_lookup_tool", tool_node)

# 7 | Fazit
---



LangGraph bietet eine Grundlage für die Entwicklung **adaptiver** und **zustandsorientierter** KI-Agenten. Durch die klare Trennung von **State**, **Knoten (Logik)** und **Kanten (Steuerung)** können Entwickler zuverlässige Workflows erstellen, deren Fokus auf dynamischen und fehlertoleranten Prozessen liegt.